In [1]:
import sys
import json
import csv 
import os
import pandas as pd

In [2]:
raw_loc = os.path.join('RawData')
clean_loc = os.path.join('static','assets','geojson')

In [3]:
big_geojson = os.path.join(raw_loc,'US_HistCounties_Gen05.json')
big_geojson

'RawData\\US_HistCounties_Gen05.json'

In [4]:
# use 1982 as a test year
year2find = 1982

# create a reference to the output file
outfile = os.path.join(raw_loc,f'Test_GeoFile{year2find}.json')

# Create a blank list of the features we will keep
features = []

# initialize a count of all records on the big geojson file
geojson_count = 0

# Open up the existing geojson file and read it into the empty geojson dictionary created above.
# While reading it in, pull the matching state and county fips from the empdata dictionary so we can add the unemployment info to the geojson.
with open(big_geojson, 'r') as f:
    temp = json.load(f)
    
    for feature in temp['features']:
        geojson_count = geojson_count + 1
        featureProperties = feature['properties']
        fips = featureProperties['FIPS']
        start_year = int(featureProperties["START_DATE"][0:4])
        end_year = int(featureProperties["END_DATE"][0:4])
        
        if start_year <= year2find and year2find <= end_year:
            features.append(feature)

print(f"Records read from large geojson file: {geojson_count}")
print(f"Records that were kept: {len(features)}")

# Output a new geojson file specific to this year.
with open(outfile,'w') as f:
    f.write('{"type":"FeatureCollection","features": [')
    for feature in features:
        json.dump(feature, f)
        f.write(',\n')
    f.write(']}')


Records read from large geojson file: 17727
Records that were kept: 3142


In [5]:
xlsfile = os.path.join(raw_loc,'GeoFRED_Per_Capita_Personal_Income_by_County_Dollars2.xls')
xl = pd.ExcelFile(xlsfile)
pcpi = xl.parse('Sheet0')
cols_in = pcpi.columns
cols_rename = []

for col in cols_in:
    if col == 'Region Code':
        cols_rename.append('FIPS')
    elif col in ('Series ID','Region Name'):
        cols_rename.append('DROP')
    elif int(col) >= 2001:
        cols_rename.append(f"PCPI_{col}")
    else:
        cols_rename.append('DROP')

pcpi.columns = cols_rename
pcpi.drop(columns =['DROP'], inplace = True) 
pcpi["In PCPI"] = 1

# Make sure the fips codes have leading zeros.
for obs in range(0,len(pcpi)):
    if len(str(pcpi.loc[obs,'FIPS'])) == 4:
        pcpi.loc[obs,'FIPS'] = f"0{pcpi.loc[obs,'FIPS']}"
    else:
        pcpi.loc[obs,'FIPS'] = f"{pcpi.loc[obs,'FIPS']}"
        
    # 46113 was renamed from Shannon County to Oglala Lakota County, SD in May 2015.  Oglala should have FIPS 46102 but
    # 46102 is not found on the big geojson file so we reset 46102 to 46113.
    if str(pcpi.loc[obs,'FIPS']) == '46102':
        pcpi.loc[obs,'FIPS'] = '46113'
        
    # Calculate percent change in PCPI
    for year in range (2002,2018):
        pyear = year - 1
        pcpi.loc[obs,f'PCPI_PctChange_{year}'] = round(100 * (pcpi.loc[obs,f'PCPI_{year}'] - pcpi.loc[obs,f'PCPI_{pyear}'])/pcpi.loc[obs,f'PCPI_{year}'],2)

pcpi.head()



FIPS  PCPI_2001  PCPI_2002  PCPI_2003  PCPI_2004  PCPI_2005  PCPI_2006  \
0  45001    23201.0    21528.0    20359.0    21756.0    22375.0    23410.0   
1  22001    21019.0    21565.0    22264.0    22569.0    24134.0    25908.0   
2  51001    21119.0    22156.0    23971.0    25337.0    26630.0    27828.0   
3  16001    33528.0    34195.0    34676.0    36657.0    37544.0    40471.0   
4  29001    21125.0    21065.0    21269.0    22361.0    23133.0    24114.0   

   PCPI_2007  PCPI_2008  PCPI_2009  ...  PCPI_PctChange_2008  \
0    24311.0    25712.0      25702  ...                 5.45   
1    27751.0    31483.0      31140  ...                11.85   
2    29914.0    32114.0      32607  ...                 6.85   
3    40561.0    39479.0      37727  ...                -2.74   
4    25295.0    26552.0      27800  ...                 4.73   

   PCPI_PctChange_2009  PCPI_PctChange_2010  PCPI_PctChange_2011  \
0                -0.04                 3.44                 3.82   
1                -1.10                 1.36                 3.74   
2                 1.51                 2.83                 1.27   
3                -4.64                 1.43                 3.47   
4                 4.49                -2.36                 4.39   

   PCPI_PctChange_2012  PCPI_PctChange_2013  PCPI_PctChange_2014  \
0                -0.44                 2.26                 4.42   
1                 4.81                 4.89                 2.96   
2                 4.32                 1.51                 3.11   
3                 4.40                 3.23                 5.31   
4                 0.74                 2.04                 2.73   

   PCPI_PctChange_2015  PCPI_PctChange_2016  PCPI_PctChange_2017  
0                 4.62                 2.70                 3.81  
1                -4.65                -4.44                 3.51  
2                 4.08                 0.75                 3.94  
3                 5.06                 1.54                 2.97  
4                 3.54                 0.15                 2.66  

[5 rows x 35 columns]

In [6]:
xlsfile = os.path.join(raw_loc,'GeoFRED_Unemployment_Rate_by_County_Percent2.xls')
xl = pd.ExcelFile(xlsfile)
df0 = xl.parse('Sheet0')
df0.drop(columns =['Series ID'], inplace = True) 
df0.head()

Region Name  Region Code  1976 January  1976 February  1976 March  \
0  Abbeville County, SC        45001           NaN            NaN         NaN   
1     Acadia Parish, LA        22001           NaN            NaN         NaN   
2   Accomack County, VA        51001           NaN            NaN         NaN   
3        Ada County, ID        16001           NaN            NaN         NaN   
4      Adair County, IA        19001           NaN            NaN         NaN   

   1976 April  1976 May  1976 June  1976 July  1976 August  ...  1996 April  \
0         NaN       NaN        NaN        NaN          NaN  ...         6.1   
1         NaN       NaN        NaN        NaN          NaN  ...         7.5   
2         NaN       NaN        NaN        NaN          NaN  ...         9.4   
3         NaN       NaN        NaN        NaN          NaN  ...         3.4   
4         NaN       NaN        NaN        NaN          NaN  ...         3.9   

   1996 May  1996 June  1996 July  1996 August  1996 September  1996 October  \
0       6.6        7.2        9.3          6.7             5.8           5.9   
1       7.6        7.9        6.9          6.7             6.8           6.4   
2       8.4        8.3        7.2          7.6             7.9           7.5   
3       3.0        3.3        3.2          3.1             3.2           3.3   
4       3.4        3.1        2.8          2.6             2.8           2.6   

   1996 November  1996 December  1997 January  
0            5.5            5.3           6.2  
1            6.2            5.6           6.1  
2            9.6           10.3          14.0  
3            3.2            3.0           3.8  
4            2.2            2.6           4.2  

[5 rows x 255 columns]

In [7]:
df1 = xl.parse('Sheet1')
df1.drop(columns =['Series ID','Region Name'], inplace = True) 
df1.head()

Region Code  1997 February  1997 March  1997 April  1997 May  1997 June  \
0        45001            9.8         9.0         8.5       8.3        6.4   
1        22001            4.9         5.2         5.1       4.9        6.2   
2        51001           13.6        12.7        10.0       8.8        9.6   
3        16001            3.6         3.3         3.0       2.6        2.9   
4        19001            3.9         3.4         3.1       2.3        2.6   

   1997 July  1997 August  1997 September  1997 October  ...  2017 May  \
0        7.4          7.4             4.4           4.0  ...       4.1   
1        5.7          8.9             9.3           4.8  ...       5.9   
2        6.4          6.9             6.8           6.2  ...       4.1   
3        2.9          2.9             3.0           3.0  ...       2.6   
4        2.2          2.3             2.4           2.1  ...       2.5   

   2017 June  2017 July  2017 August  2017 September  2017 October  \
0        4.9        5.4          5.2             4.5           4.3   
1        6.6        6.2          6.2             5.7           5.1   
2        4.0        4.1          4.3             3.9           3.9   
3        2.7        2.7          2.6             2.5           2.4   
4        2.5        2.3          2.5             2.5           2.0   

   2017 November  2017 December  2018 January  2018 February  
0            4.2            4.3           5.1            4.3  
1            4.9            4.6           5.5            5.0  
2            3.9            4.5           5.2            5.2  
3            2.7            2.6           3.3            3.1  
4            2.0            2.7           3.6            3.7  

[5 rows x 254 columns]

In [8]:
df2 = xl.parse('Sheet2')
df2.drop(columns =['Series ID','Region Name'], inplace = True) 
df2.head()

Region Code  2018 March  2018 April  2018 May  2018 June  2018 July  \
0        45001         3.9         3.1       3.3        4.4        4.3   
1        22001         5.1         4.9       5.2        6.7        6.1   
2        51001         4.6         3.5       3.4        3.6        3.5   
3        16001         2.9         2.5       2.2        2.3        2.3   
4        19001         3.0         2.6       2.0        2.2        2.0   

   2018 August  2018 September  2018 October  2018 November  2018 December  \
0          4.3             3.9           3.9            3.5            3.8   
1          5.8             5.4           5.0            4.8            4.8   
2          3.6             3.2           3.1            3.1            3.8   
3          2.2             2.1           2.1            2.3            2.4   
4          2.1             1.9           1.7            1.7            2.1   

   2019 January  2019 February  2019 March  2019 April  2019 May  2019 June  \
0           4.4            3.6         3.8         3.3       3.9        4.2   
1           5.7            4.8         4.2         3.9       4.6        5.8   
2           5.1            4.8         4.0         3.3       3.3        3.2   
3           3.3            2.9         2.7         2.2       2.1        2.4   
4           2.9            2.9         2.7         2.0       1.9        2.1   

   2019 July  
0        3.8  
1        5.5  
2        3.1  
3        2.5  
4        2.1

In [9]:
unemp = pd.merge(df0,pd.merge(df1,df2,on='Region Code'),on='Region Code')
unemp.head()

Region Name  Region Code  1976 January  1976 February  1976 March  \
0  Abbeville County, SC        45001           NaN            NaN         NaN   
1     Acadia Parish, LA        22001           NaN            NaN         NaN   
2   Accomack County, VA        51001           NaN            NaN         NaN   
3        Ada County, ID        16001           NaN            NaN         NaN   
4      Adair County, IA        19001           NaN            NaN         NaN   

   1976 April  1976 May  1976 June  1976 July  1976 August  ...  2018 October  \
0         NaN       NaN        NaN        NaN          NaN  ...           3.9   
1         NaN       NaN        NaN        NaN          NaN  ...           5.0   
2         NaN       NaN        NaN        NaN          NaN  ...           3.1   
3         NaN       NaN        NaN        NaN          NaN  ...           2.1   
4         NaN       NaN        NaN        NaN          NaN  ...           1.7   

   2018 November  2018 December  2019 January  2019 February  2019 March  \
0            3.5            3.8           4.4            3.6         3.8   
1            4.8            4.8           5.7            4.8         4.2   
2            3.1            3.8           5.1            4.8         4.0   
3            2.3            2.4           3.3            2.9         2.7   
4            1.7            2.1           2.9            2.9         2.7   

   2019 April  2019 May  2019 June  2019 July  
0         3.3       3.9        4.2        3.8  
1         3.9       4.6        5.8        5.5  
2         3.3       3.3        3.2        3.1  
3         2.2       2.1        2.4        2.5  
4         2.0       1.9        2.1        2.1  

[5 rows x 525 columns]

In [10]:
cols_in = unemp.columns
print('Original Column Names:')
print(cols_in)
cols_rename = []

for col in cols_in:
    
    if col == 'Region Code':
        cols_rename.append('FIPS')
    elif col in ('Series ID','Region Name') or col[0:3] in ('198','197'):
        cols_rename.append('DROP')
    elif "January" in col:
        cols_rename.append(f"Unemp_{col[0:4]}01")
    elif "February" in col:
        cols_rename.append(f"Unemp_{col[0:4]}02")
    elif "March" in col:
        cols_rename.append(f"Unemp_{col[0:4]}03")
    elif "April" in col:
        cols_rename.append(f"Unemp_{col[0:4]}04")
    elif "May" in col:
        cols_rename.append(f"Unemp_{col[0:4]}05")
    elif "June" in col:
        cols_rename.append(f"Unemp_{col[0:4]}06")
    elif "July" in col:
        cols_rename.append(f"Unemp_{col[0:4]}07")
    elif "August" in col:
        cols_rename.append(f"Unemp_{col[0:4]}08")
    elif "September" in col:
        cols_rename.append(f"Unemp_{col[0:4]}09")
    elif "October" in col:
        cols_rename.append(f"Unemp_{col[0:4]}10")
    elif "November" in col:
        cols_rename.append(f"Unemp_{col[0:4]}11")
    elif "December" in col:
        cols_rename.append(f"Unemp_{col[0:4]}12")
    else:
        cols_rename.append(col)

print('Renamed Column Names:')
print(cols_rename)

# Reset the column names
unemp.columns = cols_rename

# Get rid of dropped columns
unemp.drop(columns =['DROP'], inplace = True) 
unemp["In Unemp"] = 1


# Make sure the fips codes have leading zeros.
for obs in range(0,len(unemp)):
    if len(str(unemp.loc[obs,'FIPS'])) == 4:
        unemp.loc[obs,'FIPS'] = f"0{unemp.loc[obs,'FIPS']}"
    else:
        unemp.loc[obs,'FIPS'] = f"{unemp.loc[obs,'FIPS']}"
        
    # 46113 was renamed from Shannon County to Oglala Lakota County, SD in May 2015.  Oglala should have FIPS 46102 but
    # 46102 is not found on the big geojson file so we reset 46102 to 46113.
    if str(unemp.loc[obs,'FIPS']) == '46102':
        unemp.loc[obs,'FIPS'] = '46113'

unemp.head()

Original Column Names:
Index(['Region Name', 'Region Code', '1976 January', '1976 February',
       '1976 March', '1976 April', '1976 May', '1976 June', '1976 July',
       '1976 August',
       ...
       '2018 October', '2018 November', '2018 December', '2019 January',
       '2019 February', '2019 March', '2019 April', '2019 May', '2019 June',
       '2019 July'],
      dtype='object', length=525)
Renamed Column Names:
['DROP', 'FIPS', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP', 'DROP

FIPS  Unemp_199001  Unemp_199002  Unemp_199003  Unemp_199004  \
0  45001           7.0           6.8           6.8           5.5   
1  22001           5.7           5.1           5.1           4.8   
2  51001           8.3           8.1           6.7           5.4   
3  16001           4.5           4.5           3.9           3.3   
4  19001           4.0           3.7           3.3           2.5   

   Unemp_199005  Unemp_199006  Unemp_199007  Unemp_199008  Unemp_199009  ...  \
0           5.5           6.4           9.3           6.4           5.7  ...   
1           5.6           7.1           9.6           8.1           7.3  ...   
2           4.4           3.7           3.1           5.2           5.0  ...   
3           3.2           3.6           3.2           3.2           3.6  ...   
4           1.6           1.9           1.9           2.1           1.8  ...   

   Unemp_201811  Unemp_201812  Unemp_201901  Unemp_201902  Unemp_201903  \
0           3.5           3.8           4.4           3.6           3.8   
1           4.8           4.8           5.7           4.8           4.2   
2           3.1           3.8           5.1           4.8           4.0   
3           2.3           2.4           3.3           2.9           2.7   
4           1.7           2.1           2.9           2.9           2.7   

   Unemp_201904  Unemp_201905  Unemp_201906  Unemp_201907  In Unemp  
0           3.3           3.9           4.2           3.8         1  
1           3.9           4.6           5.8           5.5         1  
2           3.3           3.3           3.2           3.1         1  
3           2.2           2.1           2.4           2.5         1  
4           2.0           1.9           2.1           2.1         1  

[5 rows x 357 columns]

In [11]:
xlsfile = os.path.join(raw_loc,'EducationByCounty2.xls')
xl = pd.ExcelFile(xlsfile)
edu = xl.parse('Education 1970 to 2017')
cols_in = edu.columns

print('Original Column Names:')
print(cols_in)
cols_rename = []

cols_in

for col in cols_in:
    if col == 'FIPS Code':
        cols_rename.append('FIPS')  
    elif col == 'State':
        cols_rename.append('State Abbr')  
    elif col == 'Area name':
        cols_rename.append('County Name')  

    elif col == 'Percent of adults with less than a high school diploma, 1970':
        cols_rename.append('1970_LTHS')
    elif col == 'Percent of adults with a high school diploma only, 1970':
        cols_rename.append('1970_HS')
    elif col == 'Percent of adults completing some college (1-3 years), 1970':
        cols_rename.append('1970_SomeColl')
    elif col == 'Percent of adults completing four years of college or higher, 1970':
        cols_rename.append('1970_GRAD')

    elif col == 'Percent of adults with less than a high school diploma, 1980':
        cols_rename.append('1980_LTHS')
    elif col == 'Percent of adults with a high school diploma only, 1980':
        cols_rename.append('1980_HS')
    elif col == 'Percent of adults completing some college (1-3 years), 1980':
        cols_rename.append('1980_SomeColl')
    elif col == 'Percent of adults completing four years of college or higher, 1980':
        cols_rename.append('1980_GRAD')
        
    elif col == 'Percent of adults with less than a high school diploma, 1990':
        cols_rename.append('1990_LTHS')
    elif col == 'Percent of adults with a high school diploma only, 1990':
        cols_rename.append('1990_HS')
    elif col == "Percent of adults completing some college or associate's degree, 1990":
        cols_rename.append('1990_SomeColl')
    elif col == "Percent of adults with a bachelor's degree or higher, 1990":
        cols_rename.append('1990_GRAD')        

    elif col == 'Percent of adults with less than a high school diploma, 2000':
        cols_rename.append('2000_LTHS')
    elif col == 'Percent of adults with a high school diploma only, 2000':
        cols_rename.append('2000_HS')
    elif col == "Percent of adults completing some college or associate's degree, 2000":
        cols_rename.append('2000_SomeColl')
    elif col == "Percent of adults with a bachelor's degree or higher, 2000":
        cols_rename.append('2000_GRAD')        

    elif col == 'Percent of adults with less than a high school diploma, 2013-17':
        cols_rename.append('2013_LTHS')
    elif col == 'Percent of adults with a high school diploma only, 2013-17':
        cols_rename.append('2013_HS')
    elif col == "Percent of adults completing some college or associate's degree, 2013-17":
        cols_rename.append('2013_SomeColl')
    elif col == "Percent of adults with a bachelor's degree or higher, 2013-17":
        cols_rename.append('2013_GRAD')        
        
    else:
        cols_rename.append('DROP')


print('Renamed Column Names:')
print(cols_rename)
        
edu.columns = cols_rename
edu.drop(columns =['DROP'], inplace = True) 
edu["In Edu"] = 1

# Make sure the fips codes have leading zeros.
for obs in range(0,len(edu)):
    if len(str(edu.loc[obs,'FIPS'])) == 4:
        edu.loc[obs,'FIPS'] = f"0{edu.loc[obs,'FIPS']}"
    else:
        edu.loc[obs,'FIPS'] = f"{edu.loc[obs,'FIPS']}"
        
    # FIPS code 0 for nation or last 3 digit 000 indicate state total
    if len(str(edu.loc[obs,'FIPS'])) == 1:
        edu.loc[obs,'FIPS'] = "DROP"
    elif edu.loc[obs,'FIPS'][2:5] == '000':
        edu.loc[obs,'FIPS'] = "DROP"

    # 46113 was renamed from Shannon County to Oglala Lakota County, SD in May 2015.  Oglala should have FIPS 46102 but
    # 46102 is not found on the big geojson file so we reset 46102 to 46113.
    if str(edu.loc[obs,'FIPS']) == '46102':
        edu.loc[obs,'FIPS'] = '46113'

# Drop state and national total lines
edu = edu.loc[edu["FIPS"] != 'DROP',:]

print('Final Column Names:')
print(edu.columns)

edu.head()


Original Column Names:
Index(['FIPS Code', 'State', 'Area name', '2003 Rural-urban Continuum Code',
       '2003 Urban Influence Code', '2013 Rural-urban Continuum Code',
       '2013 Urban Influence Code', 'Less than a high school diploma, 1970',
       'High school diploma only, 1970', 'Some college (1-3 years), 1970',
       'Four years of college or higher, 1970',
       'Percent of adults with less than a high school diploma, 1970',
       'Percent of adults with a high school diploma only, 1970',
       'Percent of adults completing some college (1-3 years), 1970',
       'Percent of adults completing four years of college or higher, 1970',
       'Less than a high school diploma, 1980',
       'High school diploma only, 1980', 'Some college (1-3 years), 1980',
       'Four years of college or higher, 1980',
       'Percent of adults with less than a high school diploma, 1980',
       'Percent of adults with a high school diploma only, 1980',
       'Percent of adults completing 

FIPS State Abbr     County Name  1970_LTHS  1970_HS  1970_SomeColl  \
2  01001         AL  Autauga County       54.8     31.1            7.7   
3  01003         AL  Baldwin County       59.4     26.7            7.4   
4  01005         AL  Barbour County       68.8     19.0            4.9   
5  01007         AL     Bibb County       73.1     19.4            3.3   
6  01009         AL   Blount County       70.5     22.7            4.1   

   1970_GRAD  1980_LTHS  1980_HS  1980_SomeColl  ...  1990_GRAD  2000_LTHS  \
2        6.4       40.6     35.2           12.1  ...       14.5       21.3   
3        6.5       39.7     33.7           14.5  ...       16.8       18.0   
4        7.3       55.1     26.1            9.7  ...       11.8       35.3   
5        4.2       59.5     29.6            6.1  ...        4.7       36.8   
6        2.7       53.9     32.4            8.4  ...        7.0       29.6   

   2000_HS  2000_SomeColl  2000_GRAD  2013_LTHS  2013_HS  2013_SomeColl  \
2     33.8           26.9       18.0       12.3     33.6           29.1   
3     29.6           29.3       23.1        9.8     27.8           31.7   
4     32.4           21.3       10.9       26.9     35.5           25.5   
5     35.7           20.4        7.1       17.9     43.9           25.0   
6     36.0           24.8        9.6       20.2     32.3           34.4   

   2013_GRAD  In Edu  
2       25.0       1  
3       30.7       1  
4       12.0       1  
5       13.2       1  
6       13.1       1  

[5 rows x 24 columns]

In [12]:
xlsfile = os.path.join(raw_loc,'GDPByCounty2.csv')
gdp = pd.read_csv(xlsfile)
gdp.head()

FIPS Countyname Postal  LineCode                             IndustryName  \
0  01001    Autauga     AL       1.0                           All Industries   
1  01001    Autauga     AL       2.0       Private goods-producing industries   
2  01001    Autauga     AL       3.0    Private services-providing industries   
3  01001    Autauga     AL       4.0    Government and government enterprises   
4  01003    Baldwin     AL       1.0                           All Industries   

   2013  2014  2015  
0  -4.4  -0.7   7.6  
1   4.4   3.9   4.7  
2  -7.2  -2.2   9.9  
3  -4.0  -1.5   0.3  
4  11.1   0.5  -4.3

In [13]:
# Keep only the all-industries line
gdp = gdp.loc[gdp["IndustryName"] == 'All Industries',:]

cols_in = gdp.columns
cols_rename = []
print('Original Column Names:')
print(cols_in)

for col in cols_in:
    
    if col == 'FIPS':
        cols_rename.append('FIPS')
    elif col == 'Countyname' :
        cols_rename.append("CountyNameGDP")
    elif col == '2013' :
        cols_rename.append("GDP_Change_2012_2013")
    elif col == '2014' :
        cols_rename.append("GDP_Change_2013_2014")
    elif col == '2015' :
        cols_rename.append("GDP_Change_2014_2015")
    else:
        cols_rename.append('DROP')
        
print('Renamed Column Names:')
print(cols_rename)

gdp.columns = cols_rename
gdp.drop(columns =['DROP'], inplace = True) 
gdp["In GDP"] = 1
gdp['CombinedCounty'] = 0

# Make sure the fips codes have leading zeros.
for row, rowvals in gdp.iterrows():
     
    if rowvals[1].find('+') > -1:
        gdp.loc[row,'CombinedCounty'] = 1

print('Final Column Names:')
print(gdp.columns)


gdp.head()

Original Column Names:
Index(['FIPS', 'Countyname', 'Postal', 'LineCode', 'IndustryName', '2013',
       '2014', '2015'],
      dtype='object')
Renamed Column Names:
['FIPS', 'CountyNameGDP', 'DROP', 'DROP', 'DROP', 'GDP_Change_2012_2013', 'GDP_Change_2013_2014', 'GDP_Change_2014_2015']
Final Column Names:
Index(['FIPS', 'CountyNameGDP', 'GDP_Change_2012_2013', 'GDP_Change_2013_2014',
       'GDP_Change_2014_2015', 'In GDP', 'CombinedCounty'],
      dtype='object')


FIPS CountyNameGDP GDP_Change_2012_2013 GDP_Change_2013_2014  \
0   01001       Autauga                 -4.4                 -0.7   
4   01003       Baldwin                 11.1                  0.5   
8   01005       Barbour                  7.5                 -3.5   
12  01007          Bibb                  5.7                 -2.8   
16  01009        Blount                  8.1                 -3.3   

   GDP_Change_2014_2015  In GDP  CombinedCounty  
0                   7.6       1               0  
4                  -4.3       1               0  
8                   6.8       1               0  
12                 -4.5       1               0  
16                 17.7       1               0

In [14]:
# Output the combination counties so we can separate them, look up county codes and append them back on
combos = gdp.loc[gdp['CombinedCounty']==1,:]
countycheck_csv = os.path.join(raw_loc,'combined_counties.csv')
countycheck_out = combos.to_csv(countycheck_csv,index=False)
combos.head()

FIPS                   CountyNameGDP GDP_Change_2012_2013  \
2196   15901                  Maui + Kalawao                  0.6   
11604  51901     Albemarle + Charlottesville                 -0.4   
11608  51903           Alleghany + Covington                  5.5   
11612  51907  Augusta, Staunton + Waynesboro                  2.3   
11616  51911            Campbell + Lynchburg                 -1.4   

      GDP_Change_2013_2014 GDP_Change_2014_2015  In GDP  CombinedCounty  
2196                   2.2                  3.4       1               1  
11604                  2.2                  2.1       1               1  
11608                 -5.0                  6.5       1               1  
11612                  0.5                  4.0       1               1  
11616                 -0.2                 -0.3       1               1

In [15]:
# pull in the spreadsheet with the combined counties separated out
combosfixed = pd.read_csv(os.path.join(raw_loc,'combined_counties_separated.csv'))
combosfixed['FIPS'] = combosfixed.FIPS.astype(object)
combosfixed.dtypes

FIPS                     object
CountyNameGDP            object
GDP_Change_2012_2013    float64
GDP_Change_2013_2014    float64
GDP_Change_2014_2015    float64
In GDP                    int64
CombinedCounty            int64
dtype: object

In [16]:
len_b4 = len(gdp)
gdp = gdp.loc[gdp['CombinedCounty']==0,:]
len_during = len(gdp)
gdp = pd.concat([gdp,combosfixed])
len_after = len(gdp)

print("Added separate entries for combined counties in GDP data")
print(f"Length before removing combinations: {len_b4}")
print(f"Length after removing combinations: {len_during}")
print(f"Length after adding fixed combinations: {len_after}")

gdp.head()

Added separate entries for combined counties in GDP data
Length before removing combinations: 3113
Length after removing combinations: 3089
Length after adding fixed combinations: 3142


FIPS CountyNameGDP GDP_Change_2012_2013 GDP_Change_2013_2014  \
0   01001       Autauga                 -4.4                 -0.7   
4   01003       Baldwin                 11.1                  0.5   
8   01005       Barbour                  7.5                 -3.5   
12  01007          Bibb                  5.7                 -2.8   
16  01009        Blount                  8.1                 -3.3   

   GDP_Change_2014_2015  In GDP  CombinedCounty  
0                   7.6       1               0  
4                  -4.3       1               0  
8                   6.8       1               0  
12                 -4.5       1               0  
16                 17.7       1               0

In [17]:
check_gdp = gdp.loc[gdp["CombinedCounty"]==1,:]
check_gdp.head(50)

FIPS     CountyNameGDP GDP_Change_2012_2013 GDP_Change_2013_2014  \
0   15009              Maui                  0.6                  2.2   
1   15005           Kalawao                  0.6                  2.2   
2   51003         Albemarle                 -0.4                  2.2   
3   51540   Charlottesville                 -0.4                  2.2   
4   51005         Alleghany                  5.5                   -5   
5   51580         Covington                  5.5                   -5   
6   51015           Augusta                  2.3                  0.5   
7   51790          Staunton                  2.3                  0.5   
8   51820        Waynesboro                  2.3                  0.5   
9   51031          Campbell                 -1.4                 -0.2   
10  51680         Lynchburg                 -1.4                 -0.2   
11  51035           Carroll                    1                 -0.4   
12  51640             Galax                    1                 -0.4   
13  51053         Dinwiddie                 -0.5                 -2.6   
14  51570  Colonial Heights                 -0.5                 -2.6   
15  51730        Petersburg                 -0.5                 -2.6   
16  51059           Fairfax                  0.1                  0.6   
17  51600      Fairfax City                  0.1                  0.6   
18  51610      Falls Church                  0.1                  0.6   
19  51069         Frederick                 -3.1                 -1.3   
20  51840        Winchester                 -3.1                 -1.3   
21  51081       Greensville                   15                  6.1   
22  51595           Emporia                   15                  6.1   
23  51089             Henry                   -1                  1.5   
24  51690      Martinsville                   -1                  1.5   
25  51095        James City                 -2.8                 -2.9   
26  51830      Williamsburg                 -2.8                 -2.9   
27  51121        Montgomery                  5.8                  4.8   
28  51750           Radford                  5.8                  4.8   
29  51143      Pittsylvania                 -0.4                 -2.8   
30  51590          Danville                 -0.4                 -2.8   
31  51149     Prince George                 -1.7                 -5.3   
32  51670          Hopewell                 -1.7                 -5.3   
33  51153    Prince William                  1.2                  2.3   
34  51683          Manassas                  1.2                  2.3   
35  51685     Manassas Park                  1.2                  2.3   
36  51161    Roanoke County                  2.8                  0.9   
37  51775             Salem                  2.8                  0.9   
38  51163        Rockbridge                 -6.9                 -0.3   
39  51530       Buena Vista                 -6.9                 -0.3   
40  51678         Lexington                 -6.9                 -0.3   
41  51165        Rockingham                  0.9                 -3.8   
42  51660      Harrisonburg                  0.9                 -3.8   
43  51175       Southampton                 -4.1                 -1.2   
44  51620          Franklin                 -4.1                 -1.2   
45  51177      Spotsylvania                  4.4                 -0.4   
46  51630    Fredericksburg                  4.4                 -0.4   
47  51191        Washington                 -6.9                -13.1   
48  51520           Bristol                 -6.9                -13.1   
49  51195              Wise                 -8.6                  0.9   

   GDP_Change_2014_2015  In GDP  CombinedCounty  
0                   3.4       1               1  
1                   3.4       1               1  
2                   2.1       1               1  
3                   2.1       1               1  
4                   6.5       1

In [18]:
all_county_info = pd.merge(pcpi,\
                  pd.merge(unemp,\
                  pd.merge(edu,gdp,on='FIPS',how='outer'),on='FIPS',how='outer'),on='FIPS',how='outer')

# Clean up county names where necessary
for obs,row in all_county_info.iterrows():

    if str(all_county_info.loc[obs,'FIPS']) == '02270':
        all_county_info.loc[obs,'County Name'] = 'Wade Hampton'
        all_county_info.loc[obs,'State Abbr'] = 'AK'
        
    if all_county_info.loc[obs,'CombinedCounty'] == 1:
        
        if str(all_county_info.loc[obs,'FIPS'])[0:2] == '15':
            all_county_info.loc[obs,'State Abbr'] = "HI"
        elif str(all_county_info.loc[obs,'FIPS'])[0:2] == '51':
            all_county_info.loc[obs,'State Abbr'] = "VA"
        all_county_info.loc[obs,'County Name'] = all_county_info.loc[obs,'CountyNameGDP'] 
        

all_county_info.head()

FIPS  PCPI_2001  PCPI_2002  PCPI_2003  PCPI_2004  PCPI_2005  PCPI_2006  \
0  45001    23201.0    21528.0    20359.0    21756.0    22375.0    23410.0   
1  22001    21019.0    21565.0    22264.0    22569.0    24134.0    25908.0   
2  51001    21119.0    22156.0    23971.0    25337.0    26630.0    27828.0   
3  16001    33528.0    34195.0    34676.0    36657.0    37544.0    40471.0   
4  29001    21125.0    21065.0    21269.0    22361.0    23133.0    24114.0   

   PCPI_2007  PCPI_2008  PCPI_2009  ...  2013_HS  2013_SomeColl  2013_GRAD  \
0    24311.0    25712.0    25702.0  ...     34.3           29.5       14.7   
1    27751.0    31483.0    31140.0  ...     40.8           24.7       12.2   
2    29914.0    32114.0    32607.0  ...     39.0           23.6       19.6   
3    40561.0    39479.0    37727.0  ...     22.4           34.7       37.6   
4    25295.0    26552.0    27800.0  ...     35.6           23.9       31.0   

   In Edu  CountyNameGDP  GDP_Change_2012_2013  GDP_Change_2013_2014  \
0     1.0      Abbeville                  -2.5                  -4.7   
1     1.0         Acadia                  -6.9                  -0.5   
2     1.0       Accomack                   2.2                  -1.0   
3     1.0            Ada                   5.9                   3.9   
4     1.0          Adair                   1.7                   1.1   

   GDP_Change_2014_2015  In GDP  CombinedCounty  
0                  -0.8     1.0             0.0  
1                  -0.3     1.0             0.0  
2                   2.2     1.0             0.0  
3                   1.3     1.0             0.0  
4                   5.7     1.0             0.0  

[5 rows x 420 columns]

In [19]:
# Find places where the county and state name are blank.
data2check = all_county_info.loc[all_county_info["State Abbr"].isnull() == True,['FIPS','FIPS Out','In PCPI',\
                                                                                 'In Edu','In GDP','In Unemp','State Abbr',\
                                                                                 'County Name','CountyNameGDP',\
                                                                                 'GDP_Change_2012_2013', \
                                                                                 'GDP_Change_2013_2014',\
                                                                                 'GDP_Change_2014_2015', ]]
print(len(data2check))
data2check.head(30)

0


C:\Users\laslo\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Empty DataFrame
Columns: [FIPS, FIPS Out, In PCPI, In Edu, In GDP, In Unemp, State Abbr, County Name, CountyNameGDP, GDP_Change_2012_2013, GDP_Change_2013_2014, GDP_Change_2014_2015]
Index: []

In [20]:
data2check2 = all_county_info.loc[all_county_info['In PCPI'] + all_county_info['In GDP'] + \
                                  all_county_info['In Edu'] + all_county_info['In Unemp'] != 4,\
                                  ['FIPS','In PCPI','In Edu','In Unemp','In GDP','State Abbr','County Name']]
print(f"There are {len(data2check2)} counties that are not in all four input datasets")
data2check2.head()

There are 200 counties that are not in all four input datasets


FIPS  In PCPI  In Edu  In Unemp  In GDP State Abbr  \
278   08014      1.0     1.0       NaN     1.0         CO   
1511  02270      1.0     NaN       1.0     NaN         AK   
2204  02195      1.0     1.0       NaN     1.0         AK   
2575  02230      1.0     1.0       NaN     1.0         AK   
3052  02275      1.0     1.0       NaN     1.0         AK   

                    County Name  
278           Broomfield County  
1511               Wade Hampton  
2204     Petersburg Census Area  
2575       Skagway Municipality  
3052  Wrangell City and Borough

In [21]:
county_csv = os.path.join(clean_loc,'CountyLevelData.csv')
county_out = all_county_info.to_csv(county_csv,index=False)

In [22]:
vnames = all_county_info.columns.tolist()

In [23]:
# Create a dictionary to hold state and county fips codes
# fipsdict has the state and county associated with each fips code
fipsdict = {}

# Go through the list of counties in our data and make sure each is 
# represented in the fipsdict. This will allow us to have FIPS, STATEFP,
# and COUNTYFP on each geojson file.
for row, rowvals in all_county_info.iterrows():
    
    # pull information from dataframe into temporary variables
    FIPS = str(rowvals[0])
    STATEFP = str(FIPS)[0:2]
    COUNTYFP = str(FIPS)[2:5]
    
    # If we have not yet seen this FIPS code, add it to the dictionary
    if FIPS not in fipsdict:
        fipsdict[FIPS]={}
 
    # Append the data for this county to its entry in the dictionary
    fipsdict[FIPS]={
        "STATEFP": STATEFP,
        "COUNTYFP": COUNTYFP}
    
fipsdict

{'45001': {'STATEFP': '45', 'COUNTYFP': '001'},
 '22001': {'STATEFP': '22', 'COUNTYFP': '001'},
 '51001': {'STATEFP': '51', 'COUNTYFP': '001'},
 '16001': {'STATEFP': '16', 'COUNTYFP': '001'},
 '29001': {'STATEFP': '29', 'COUNTYFP': '001'},
 '40001': {'STATEFP': '40', 'COUNTYFP': '001'},
 '19001': {'STATEFP': '19', 'COUNTYFP': '001'},
 '21001': {'STATEFP': '21', 'COUNTYFP': '001'},
 '53001': {'STATEFP': '53', 'COUNTYFP': '001'},
 '39001': {'STATEFP': '39', 'COUNTYFP': '001'},
 '42001': {'STATEFP': '42', 'COUNTYFP': '001'},
 '16003': {'STATEFP': '16', 'COUNTYFP': '003'},
 '28001': {'STATEFP': '28', 'COUNTYFP': '001'},
 '38001': {'STATEFP': '38', 'COUNTYFP': '001'},
 '19003': {'STATEFP': '19', 'COUNTYFP': '003'},
 '31001': {'STATEFP': '31', 'COUNTYFP': '001'},
 '18001': {'STATEFP': '18', 'COUNTYFP': '001'},
 '55001': {'STATEFP': '55', 'COUNTYFP': '001'},
 '17001': {'STATEFP': '17', 'COUNTYFP': '001'},
 '08001': {'STATEFP': '08', 'COUNTYFP': '001'},
 '50001': {'STATEFP': '50', 'COUNTYFP': 

In [24]:
# Create the initial geojson file by pulling records from the big/historical geojson
# and keeping only records that apply to that year.
def create_yearly_geojson(year):
    
    # create a reference to the output file
    outfile = os.path.join(clean_loc,f'FinalGeoFile{year}.json')

    # Create a numeric variable of the year to check against
    year2check = year
    
    # Create a blank list of the features we will keep
    features = []
    
    # Open up the existing geojson file and read it into the empty geojson dictionary created above.
    # While reading it in, pull the matching state and county fips from the data2add dictionary so we can add the unemployment info to the geojson.
    with open(big_geojson, 'r') as f:
        temp = json.load(f)
        
        for feature in temp['features']:
            
            featureProperties = feature['properties']
            FIPS = featureProperties['FIPS']
            
            start_year = int(featureProperties["START_DATE"][0:4])
            end_year = int(featureProperties["END_DATE"][0:4])
            
            # Add the state and county fips codes so this json will look
            # like the ones for later years from Census
            featureData = fipsdict.get(FIPS, {})
            for key in featureData.keys():
                featureProperties[key] = featureData[key]
        
            if start_year <= year2check and year2check <= end_year:
                features.append(feature)
                
    last_feature = len(features)-1
                
    # Output a new geojson file specific to this year.
    with open(outfile,'w') as f:
        f.write('{"type":"FeatureCollection","features": [')
        for feature in features:
            json.dump(feature, f)
            if features.index(feature) < last_feature:
                f.write(',')
        f.write(']}')

create_yearly_geojson(1970)
create_yearly_geojson(1980)
for year in range(1990,2001):
    create_yearly_geojson(year)  


In [46]:

def create_final_geojson(year):
    
    year2check = year
    pyear = year - 1

    # for years greater than 2000 we also need the two digit year
    yr = str(year2check)[2:4]
    
    # create a reference to the input geojson file. Different years have different files/formats
    if year2check <= 2000:
        infile = os.path.join(clean_loc,f'FinalGeoFile{year}.json')
    elif year2check <= 2009:
        infile = os.path.join(clean_loc,f'FinalGeoFile2000.json')
    #elif year2check <= 2009:
    #    infile = os.path.join(raw_loc,f'tl_{year}_us_county00.json')
    #elif year2check == 2010:
    #    infile = os.path.join(raw_loc,f'tl_2010_us_county10.json')
    #elif year2check <= 2012:
    #    infile = os.path.join(raw_loc,f'tl_{year}_us_county.json')
    elif year2check <= 2013:
        infile = os.path.join(raw_loc,f'cb_2013_us_county_20m.json')
    elif year2check <= 2018:
        infile = os.path.join(raw_loc,f'cb_{year}_us_county_20m.json')
    #elif year2check <= 2019:
    #    infile = os.path.join(raw_loc,f'tl_{year}_us_county.json')
    else:
        infile = os.path.join(raw_loc,f'cb_2018_us_county_20m.json')
    
    # create a reference to the output file
    outfile = os.path.join(clean_loc,f'FinalGeoFile{year}.json')
    
    print(f"infile = {infile}")
    print(f"outfile = {outfile}")
    
    # Create a blank dictionary to hold the data we will be tacking onto the geojson file
    data2add = {}

    # Loop through the dataframe and add information to the data2add dictionary. 
    # We will use this to put these values into the geojson.
    for row, rowvals in all_county_info.iterrows():
        
        # pull information from dataframe into temporary variables
        FIPS = rowvals[0]   
        STATEFP = str(rowvals[0])[0:2]
        COUNTYFP = str(rowvals[0])[2:5]
        StateAbbr = rowvals[vnames.index('State Abbr')]  
        CountyName = rowvals[vnames.index('County Name')]
        
        # pull monthly unemployment for this year, if it exists for this year
        if f'Unemp_{year}01' in vnames:
            temp01 = rowvals[vnames.index(f'Unemp_{year}01')]
        else:
            temp01 = -1
        if f'Unemp_{year}02' in vnames:
            temp02 = rowvals[vnames.index(f'Unemp_{year}02')]
        else:
            temp02 = -1
        if f'Unemp_{year}03' in vnames:
            temp03 = rowvals[vnames.index(f'Unemp_{year}03')]
        else:
            temp03 = -1
        if f'Unemp_{year}04' in vnames:
            temp04 = rowvals[vnames.index(f'Unemp_{year}04')]
        else:
            temp04 = -1
        if f'Unemp_{year}05' in vnames:
            temp05 = rowvals[vnames.index(f'Unemp_{year}05')]
        else:
            temp05 = -1
        if f'Unemp_{year}06' in vnames:
            temp06 = rowvals[vnames.index(f'Unemp_{year}06')]
        else:
            temp06 = -1
        if f'Unemp_{year}07' in vnames:
            temp07 = rowvals[vnames.index(f'Unemp_{year}07')]
        else:
            temp07 = -1
        if f'Unemp_{year}08' in vnames:
            temp08 = rowvals[vnames.index(f'Unemp_{year}08')]
        else:
            temp08 = -1
        if f'Unemp_{year}09' in vnames:
            temp09 = rowvals[vnames.index(f'Unemp_{year}09')]
        else:
            temp09 = -1
        if f'Unemp_{year}10' in vnames:
            temp10 = rowvals[vnames.index(f'Unemp_{year}10')]
        else:
            temp10 = -1
        if f'Unemp_{year}11' in vnames:
            temp11 = rowvals[vnames.index(f'Unemp_{year}11')]
        else:
            temp11 = -1
        if f'Unemp_{year}12' in vnames:
            temp12 = rowvals[vnames.index(f'Unemp_{year}12')]
        else:
            temp12 = -1

        # Pull per-capital personal income, if it exists for this year
        if f'PCPI_{pyear}' in vnames and f'PCPI_{year}' in vnames:
            PCPI = rowvals[vnames.index(f'PCPI_{year}')]
            PCPI_previous = rowvals[vnames.index(f'PCPI_{pyear}')]
            PCPI_PctChange = rowvals[vnames.index(f'PCPI_PctChange_{year}')]
        else:
            PCPI = -1
            PCPI_previous = -1
            PCPI_PctChange = -1
            
        # Pull gdp, if it exists for this year
        if year2check == 2013:
            GDP = rowvals[vnames.index('GDP_Change_2012_2013')]
        elif year2check == 2014:
            GDP = rowvals[vnames.index('GDP_Change_2013_2014')]
        elif year2check == 2015:
            GDP = rowvals[vnames.index('GDP_Change_2014_2015')]
        else:
            GDP = -1

        # Pull eductional attainment, if it exists for this year
        if year2check in (1970, 1980, 1990, 2000, 2013):
            LTHS = rowvals[vnames.index(f'{year}_LTHS')]
            HS = rowvals[vnames.index(f'{year}_HS')]
            SomeColl = rowvals[vnames.index(f'{year}_SomeColl')]
            GRAD = rowvals[vnames.index(f'{year}_GRAD')]
        else:
            LTHS = -1
            HS = -1
            SomeColl = -1
            GRAD = -1
            
        # Check to see if this state/county has already been added to the dictionary.
        # If not, then add an entry for it.
        if FIPS not in data2add:
            data2add[FIPS]={}
        
        # Append the data for this county to its entry in the dictionary
        data2add[FIPS]={
            "StateAbbr": StateAbbr,
            "CountyName": CountyName,
            f"Unemp_{year}01" : f"{temp01}",
            f"Unemp_{year}02" : f"{temp02}",
            f"Unemp_{year}03" : f"{temp03}",
            f"Unemp_{year}04" : f"{temp04}",
            f"Unemp_{year}05" : f"{temp05}",
            f"Unemp_{year}06" : f"{temp06}",
            f"Unemp_{year}07" : f"{temp07}",
            f"Unemp_{year}08" : f"{temp08}",
            f"Unemp_{year}09" : f"{temp09}",
            f"Unemp_{year}10" : f"{temp10}",
            f"Unemp_{year}11" : f"{temp11}",
            f"Unemp_{year}12" : f"{temp12}",
            f"PCPI_Current": f"{PCPI}",
            f"PCPI_Previous": f"{PCPI_previous}",
            f"PCPI_PctChange": f"{PCPI_PctChange}",
            f"GDP_change_{year}": f"{GDP}",
            "LTHS" : f"{LTHS}",
            "HS" : f"{HS}",
            "SomeColl" : f"{SomeColl}",
            "GRAD" : f"{GRAD}"
        }
    
    #print(data2add)
            
    # Create a blank geojson for the final result
    geojson = {}
    
    # Open up the existing geojson file and read it into the empty geojson dictionary created above.
    # While reading it in, pull the matching state and county fips from the data2add dictionary so we can add the unemployment info to the geojson.
    with open(infile, 'r') as f:
        geojson = json.load(f)
        for feature in geojson['features']:
            featureProperties = feature['properties']
            
            if year2check <= 2009:
                FIPS = featureProperties['FIPS']
                STATEFP = str(FIPS)[0:2]
                COUNTYFP = str(FIPS)[2:5]
            elif year2check <= 2020:
                STATEFP = featureProperties['STATEFP']
                COUNTYFP = featureProperties['COUNTYFP']
                FIPS = STATEFP + COUNTYFP

            featureData = data2add.get(FIPS, {})
            for key in featureData.keys():
                featureProperties[key] = featureData[key]

    # Output this updated geojson.
    with open(outfile, 'w') as f:
        json.dump(geojson, f)
         
#create_final_geojson(1970)
#create_final_geojson(1980)
#for year in range(1990,2020):
#    create_final_geojson(year)

create_final_geojson(2019)

infile = RawData\cb_2018_us_county_20m.json
outfile = static\assets\geojson\FinalGeoFile2019.json


In [26]:
# print out statements that we can paste into javascript

    

In [27]:
def printlinks(year):
    print(f'countyLink{year} = "/static/assets/geojson/FinalGeoFile{year}.json";')
    
printlinks(1970)
printlinks(1980)
for year in range(1990,2020):
    printlinks(year)

countyLink1970 = "/static/assets/geojson/FinalGeoFile1970.json";
countyLink1980 = "/static/assets/geojson/FinalGeoFile1980.json";
countyLink1990 = "/static/assets/geojson/FinalGeoFile1990.json";
countyLink1991 = "/static/assets/geojson/FinalGeoFile1991.json";
countyLink1992 = "/static/assets/geojson/FinalGeoFile1992.json";
countyLink1993 = "/static/assets/geojson/FinalGeoFile1993.json";
countyLink1994 = "/static/assets/geojson/FinalGeoFile1994.json";
countyLink1995 = "/static/assets/geojson/FinalGeoFile1995.json";
countyLink1996 = "/static/assets/geojson/FinalGeoFile1996.json";
countyLink1997 = "/static/assets/geojson/FinalGeoFile1997.json";
countyLink1998 = "/static/assets/geojson/FinalGeoFile1998.json";
countyLink1999 = "/static/assets/geojson/FinalGeoFile1999.json";
countyLink2000 = "/static/assets/geojson/FinalGeoFile2000.json";
countyLink2001 = "/static/assets/geojson/FinalGeoFile2001.json";
countyLink2002 = "/static/assets/geojson/FinalGeoFile2002.json";
countyLink2003 = "/static

In [28]:

def printvars(year):
    print(f"var countyLayer{year} = new L.LayerGroup();")
    
printvars(1970)
printvars(1980)
for year in range(1990,2020):
    printvars(year)


var countyLayer1970 = new L.LayerGroup();
var countyLayer1980 = new L.LayerGroup();
var countyLayer1990 = new L.LayerGroup();
var countyLayer1991 = new L.LayerGroup();
var countyLayer1992 = new L.LayerGroup();
var countyLayer1993 = new L.LayerGroup();
var countyLayer1994 = new L.LayerGroup();
var countyLayer1995 = new L.LayerGroup();
var countyLayer1996 = new L.LayerGroup();
var countyLayer1997 = new L.LayerGroup();
var countyLayer1998 = new L.LayerGroup();
var countyLayer1999 = new L.LayerGroup();
var countyLayer2000 = new L.LayerGroup();
var countyLayer2001 = new L.LayerGroup();
var countyLayer2002 = new L.LayerGroup();
var countyLayer2003 = new L.LayerGroup();
var countyLayer2004 = new L.LayerGroup();
var countyLayer2005 = new L.LayerGroup();
var countyLayer2006 = new L.LayerGroup();
var countyLayer2007 = new L.LayerGroup();
var countyLayer2008 = new L.LayerGroup();
var countyLayer2009 = new L.LayerGroup();
var countyLayer2010 = new L.LayerGroup();
var countyLayer2011 = new L.LayerG

In [29]:

def printmonthvars(year):
    for i in range(12):
        m = i + 1
        if m < 10:
            mm = f'0{m}'
        else:
            mm = m
        yymm = str(year) + str(mm)
        print(f"var countyLayer{yymm} = new L.LayerGroup();")
        print(f"monthLayers({year},{mm},countyLink{year},countyLayer{year}{mm});")
        print(f"layers.push(countyLayer{year}{mm});")
        print(f"descrs.push({year}{mm});")
for year in range(1990,2020):
    printmonthvars(year)

var countyLayer199001 = new L.LayerGroup();
monthLayers(1990,01,countyLink1990,countyLayer199001);
layers.push(countyLayer199001);
descrs.push(199001);
var countyLayer199002 = new L.LayerGroup();
monthLayers(1990,02,countyLink1990,countyLayer199002);
layers.push(countyLayer199002);
descrs.push(199002);
var countyLayer199003 = new L.LayerGroup();
monthLayers(1990,03,countyLink1990,countyLayer199003);
layers.push(countyLayer199003);
descrs.push(199003);
var countyLayer199004 = new L.LayerGroup();
monthLayers(1990,04,countyLink1990,countyLayer199004);
layers.push(countyLayer199004);
descrs.push(199004);
var countyLayer199005 = new L.LayerGroup();
monthLayers(1990,05,countyLink1990,countyLayer199005);
layers.push(countyLayer199005);
descrs.push(199005);
var countyLayer199006 = new L.LayerGroup();
monthLayers(1990,06,countyLink1990,countyLayer199006);
layers.push(countyLayer199006);
descrs.push(199006);
var countyLayer199007 = new L.LayerGroup();
monthLayers(1990,07,countyLink1990,countyLay

In [35]:

def printyearvars(year):
        print(f"var countyLayer{year} = new L.LayerGroup();")
        print(f"yearLayers({year},countyLink{year},countyLayer{year});")
        print(f"layers.push(countyLayer{year});")
        print(f"descrs.push({year});")
for year in range(1990,2020):
    printyearvars(year)

var countyLayer1990 = new L.LayerGroup();
yearLayers(1990,countyLink1990,countyLayer1990);
layers.push(countyLayer1990);
descrs.push(1990);
var countyLayer1991 = new L.LayerGroup();
yearLayers(1991,countyLink1991,countyLayer1991);
layers.push(countyLayer1991);
descrs.push(1991);
var countyLayer1992 = new L.LayerGroup();
yearLayers(1992,countyLink1992,countyLayer1992);
layers.push(countyLayer1992);
descrs.push(1992);
var countyLayer1993 = new L.LayerGroup();
yearLayers(1993,countyLink1993,countyLayer1993);
layers.push(countyLayer1993);
descrs.push(1993);
var countyLayer1994 = new L.LayerGroup();
yearLayers(1994,countyLink1994,countyLayer1994);
layers.push(countyLayer1994);
descrs.push(1994);
var countyLayer1995 = new L.LayerGroup();
yearLayers(1995,countyLink1995,countyLayer1995);
layers.push(countyLayer1995);
descrs.push(1995);
var countyLayer1996 = new L.LayerGroup();
yearLayers(1996,countyLink1996,countyLayer1996);
layers.push(countyLayer1996);
descrs.push(1996);
var countyLayer1997 

In [44]:
def print1monthper(year):
    print(f"var countyLayer{year}07 = new L.LayerGroup();")
    print(f"monthLayers({year},07,countyLink{year},countyLayer{year}07);")
    print(f"layers.push(countyLayer{year}07);")
    print(f"descrs.push({year});")
    
for year in range(1990,2020):
    
    if year <= 2005 and (year % 5) == 0:
        print1monthper(year)
    elif year > 2005 and year <= 2013 and (year % 3) == 0:
        print1monthper(year)
    elif year > 2013:
        print1monthper(year)

var countyLayer199007 = new L.LayerGroup();
monthLayers(1990,07,countyLink1990,countyLayer199007);
layers.push(countyLayer199007);
descrs.push(1990);
var countyLayer199507 = new L.LayerGroup();
monthLayers(1995,07,countyLink1995,countyLayer199507);
layers.push(countyLayer199507);
descrs.push(1995);
var countyLayer200007 = new L.LayerGroup();
monthLayers(2000,07,countyLink2000,countyLayer200007);
layers.push(countyLayer200007);
descrs.push(2000);
var countyLayer200507 = new L.LayerGroup();
monthLayers(2005,07,countyLink2005,countyLayer200507);
layers.push(countyLayer200507);
descrs.push(2005);
var countyLayer200707 = new L.LayerGroup();
monthLayers(2007,07,countyLink2007,countyLayer200707);
layers.push(countyLayer200707);
descrs.push(2007);
var countyLayer201007 = new L.LayerGroup();
monthLayers(2010,07,countyLink2010,countyLayer201007);
layers.push(countyLayer201007);
descrs.push(2010);
var countyLayer201307 = new L.LayerGroup();
monthLayers(2013,07,countyLink2013,countyLayer201307);
l